# Milestone 1: Advertising

**Business Context:** 

Retail company, "Fashion Haven," operates multiple stores in different cities. The company invests in advertising campaigns to promote its latest collections through various media sources like TV, Newspaper, and Radio. They want to understand the impact of each media source on their sales revenue to optimize their advertising strategy and improve overall business performance.

Currently, Fashion Haven lacks an effective method to predict the sales revenue generated from their advertising efforts accurately. As a result, they struggle to allocate their advertising budget optimally across different media channels, leading to sub optimal returns on investment and inefficient resource allocation.

To address this business problem, Fashion Haven has collected historical data containing information on various advertising campaigns (TV, Newspaper, Radio) and their corresponding sales revenue across their different store locations. The goal is to build a robust predictive model that accurately estimates the sales revenue based on the media sources' advertising budgets, helping the company make data-driven decisions and drive business growth.


Dataset Description:

The data contains the different attributes of the advertising business. The detailed data dictionary is given below.

* TV: Expenditure on media resource- TV 
* Radio: Expenditure on media resource- Radio 
* NewsPaper: Expenditure on media resource- Newspaper 
* Sales: Target Column - Amount of Sales

### ** Connect to the Workspace **  

In [1]:
# Handle to the workspace
from azure.ai.ml import MLClient

# Authentication package
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()

In [2]:
# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id="6793e723-756c-4c5d-84c0-812f1bb4c679", # Subscription ID
    resource_group_name="JuvlinResourceGroup",              # Resource Group
    workspace_name="JuvlinWorkspace",
)

### ** Upload the dataset on Blob Storage as Data Asset **

In [3]:
# Use Data Asset created from File uploaded to Blob Storage
data_asset = ml_client.data.get("advertising_data", version="1")
da_path=data_asset.path								
print(da_path)		

azureml://subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourcegroups/JuvlinResourceGroup/workspaces/JuvlinWorkspace/datastores/workspaceblobstore/paths/UI/2024-11-09_202832_UTC/advertising_raw.csv/


### ** Create a Compute Resource to run the Jobs **

In [4]:
from azure.ai.ml.entities import AmlCompute

# Assign Name to the compute cluster
cpu_compute_target = "cpu-cluster-E2E"

# Define the Azure ML compute object with the intended parameters
cpu_cluster = AmlCompute(
    name=cpu_compute_target,
    type="amlcompute",                 # Azure ML Compute is the on-demand VM service
    size="STANDARD_D2_V3",             # VM Family
    min_instances=0,                   # Minimum running nodes when there is no job running
    max_instances=1,                   # Nodes in cluster
    idle_time_before_scale_down=180,   # How many seconds will the node running after the job termination
    tier="Dedicated",                  # Dedicated or LowPriority. The latter is cheaper but there is a chance of job termination
)

try:
    cpu_cluster = ml_client.compute.get(cpu_compute_target)  # Check if the compute target already exists
    print(
        f"Compute Cluster {cpu_compute_target} exists, reuse it as is."
    )
except Exception:
    print("Creating a new Compute Cluster...") 
    cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster).result()   # Pass the object to MLClient's create_or_update method
    print(
    f"AMLCompute Cluster {cpu_cluster.name} created, the compute size is {cpu_cluster.size}"
)

Compute Cluster cpu-cluster-E2E exists, reuse it as is.


### ** Create a Custom Job Environment **

In [5]:
import os

# Name of the directory to create
dependencies_dir = "./env"

# Make directory, if exists don't raise an exception 
os.makedirs(dependencies_dir, exist_ok=True)

** Create YAML File with required dependencies **

-  ** 1. Create and Register the Custom Job Environment in the Workspace. **
-  ** 2. The Environment will be packaged into a Docker Container at runtime. **

In [6]:
%%writefile {dependencies_dir}/conda1.yaml

name: sklearn-env
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pip=21.2.4
  - scikit-learn=1.0.2
  - scipy=1.7.1
  - pandas
  - numpy
  - pip:
      - mlflow==2.8.1
      - azureml-mlflow==1.51.0
      - azureml-core==1.51.0  
      - azureml-inference-server-http
      - cloudpickle==1.6.0

Overwriting ./env/conda1.yaml


** Create Environment for ML Tasks **

In [7]:
# Import the Environment Class from the azure.ai.ml.entities module
from azure.ai.ml.entities import Environment


# Name of the Custom Environment to create
custom_env_name = "Milestone1_Project_E2E"

# Create Environment object with the specified properties
job_env = Environment(
    name=custom_env_name,
    description="Custom Environment for ML task",
    conda_file=os.path.join(dependencies_dir, "conda1.yaml"),
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)
job_env = ml_client.environments.create_or_update(job_env)

# Print information for registered environment
print(
    f"Environment {job_env.name} registered to workspace, the environment version is {job_env.version}"
)

Environment Milestone1_Project_E2E registered to workspace, the environment version is 9


### ** Exploratory Data Analysis **

In [8]:
# Import required Libraries
import argparse
import os
import numpy as np
import pandas as pd
from sklearn import datasets
import mlflow
import mlflow.sklearn

df = pd.read_csv(da_path)

In [9]:
# Parse Features to the correct datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


In [10]:
# Table Summary
df.describe()

,TV,Radio,Newspaper,Sales
count,200.000000,200.000000,200.000000,200.000000
mean,147.042500,23.264000,30.554000,14.022500
std,85.854236,14.846809,21.778621,5.217457
min,0.700000,0.000000,0.300000,1.600000
25%,74.375000,9.975000,12.750000,10.375000
50%,149.750000,22.900000,25.750000,12.900000
75%,218.825000,36.525000,45.100000,17.400000
max,296.400000,49.600000,114.000000,27.000000


In [11]:
# Display the first 5 record
df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [12]:
# Display the last 5 record
df.tail()

,TV,Radio,Newspaper,Sales
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5
199,232.1,8.6,8.7,13.4


In [13]:
# Check for missing values
df.isnull().sum()

TV           0
Radio        0
Newspaper    0
Sales        0
dtype: int64

** Conclusion: There are no missing or null values **

In [14]:
# Check for all duplicate rows
df[df.duplicated()]

,TV,Radio,Newspaper,Sales


** Conclusion: There are no duplicate records **

### EDA Conclusion: Data is clean and no additional preprocessing is needed ###

### ** Create a Training Script to perform the Training Job **

In [15]:
# Create Directory to store Training Script file
import os

src_dir = "./src"
os.makedirs(src_dir, exist_ok=True)

In [16]:
%%writefile {src_dir}/milestone1_training.py

# Import required Libraries
import argparse
import os
import numpy as np
import pandas as pd

from sklearn import datasets
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

import mlflow
import mlflow.sklearn

# Create an argument parser for input arguments from command line
def main():
    parser = argparse.ArgumentParser()

    parser.add_argument("--data", type=str, help="path to input data")
    parser.add_argument('--criterion', type=str, default='mse', help='measure the quality of a split')
    parser.add_argument('--max-depth', type=int, default=None, help='The maximum depth of the tree.')
    parser.add_argument("--test_train_ratio", type=float, required=False, default=0.25)
    parser.add_argument("--registered_model_name", type=str, help="model name")

    args = parser.parse_args()

    # Start Logging
    mlflow.start_run()

    # Enable AutoLogging
    mlflow.sklearn.autolog()

    # Print Input Arguments
    print(" ".join(f"{k}={v}" for k, v in vars(args).items()))

    # Load Input Data
    print("input data:", args.data)
    df = pd.read_csv(args.data)

    # Log input hyperparameters
    mlflow.log_param('Criterion', str(args.criterion))
    mlflow.log_param('Max depth', str(args.max_depth))

    # Split data into features (X) and target (y)
    X = df[['TV', 'Radio', 'Newspaper']]
    y = df['Sales']
    
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=args.test_train_ratio, random_state=42)
    
    # Initialize and Train a Decision Tree Regressor

    tree_model = DecisionTreeRegressor(criterion=args.criterion, max_depth=args.max_depth, random_state=42)
    tree_model = tree_model.fit(X_train, y_train)
    tree_predictions = tree_model.predict(X_test)

    # Compute and Log Model for MSE metric
    mse = mean_squared_error(y_test, tree_predictions)
    print(f"Mean Squared Error on test set: {mse}")
    mlflow.log_metric('Mean Squared Error', float(mse))

    # Set Name for the Registered Model
    registered_model_name="milestone1_model"

    # Register Model to the Workspace
    print("Registering the model via MLFlow")
    mlflow.sklearn.log_model(
        sk_model=tree_model,
        registered_model_name=registered_model_name,
        artifact_path=registered_model_name
    )

    # Save Model to a File
    print("Saving the model via MLFlow")
    mlflow.sklearn.save_model(
        sk_model=tree_model,
        path=os.path.join(registered_model_name, 'trained_model'),
    )
   
    # End MLflow tracking
    mlflow.end_run()

if __name__ == '__main__':
    main()


Overwriting ./src/milestone1_training.py


### ** Configure the Training Job **

In [17]:
# Import required Modules

from azure.ai.ml import command
from azure.ai.ml import Input

# Define a new AML job using the Command function
job = command(
    inputs=dict(
        data=Input(
            type="uri_file",           
            path=da_path,                      # Path to the input data file        
        ),
        test_train_ratio=0.3,                  # Ratio of the data to be used for testing
        criterion="mse",                       # Criterion used to measure the quality of a split
        max_depth=2,                           # Maximum depth of the Decision Tree
    ),
    
    code="./src/",                             # Specify Directory containing the code to be run in the job
 
    # Specify command to run in the job, including the input data and parameters as command line arguments
    command="python milestone1_training.py --data ${{inputs.data}} --test_train_ratio ${{inputs.test_train_ratio}} --criterion ${{inputs.criterion}} --max-depth ${{inputs.max_depth}}",
 
    environment="Milestone1_Project_E2E@latest",     # Specify environment or job
    compute="cpu-cluster-E2E",                       # Specify compute target for job
    experiment_name="milestone1_train_prediction",   # Specify experiment name for job 
    display_name="milestone1_prediction",            # Specify display name for job
)

### ** Run the Training Job **

In [18]:
# Create new Job or update using ml_client.create_or_update
training_job = ml_client.create_or_update(job)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


In [19]:
# Wait for the job to complete by streaming logs
ml_client.jobs.stream(training_job.name)

# Proceed with the next steps after the job is complete
print("Job has completed. Proceeding to the next step.")

RunId: gentle_giraffe_kwb8mw9cx7
Web View: https://ml.azure.com/runs/gentle_giraffe_kwb8mw9cx7?wsid=/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourcegroups/JuvlinResourceGroup/workspaces/JuvlinWorkspace

Execution Summary
RunId: gentle_giraffe_kwb8mw9cx7
Web View: https://ml.azure.com/runs/gentle_giraffe_kwb8mw9cx7?wsid=/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourcegroups/JuvlinResourceGroup/workspaces/JuvlinWorkspace

Job has completed. Proceeding to the next step.


### ** Define the parameter space for Hyperparameter Tuning **

In [20]:
from azure.ai.ml.sweep import Choice

# Reuse the command_job created 
job_for_sweep = job(
    criterion=Choice(values=["mse"]),
    max_depth=Choice(values=[2, 3, 4, 5]),
)

### ** Configure the Sweep Job for Tuning **

- ** compute ** - specifies the compute target where the sweep job will run.
- ** sampling_algorithm ** - specifies the search algorithm to use for hyperparameter tuning.
- ** primary_metric ** - specifies the metric to optimize during hyperparameter tuning.
- ** goal ** - specifies whether to maximize or minimize the primary metric.
- ** max_total_trials ** - specifies the maximum number of trials to run during hyperparameter tuning.
- ** max_concurrent_trials ** - specifies the maximum number of trials to run concurrently during hyperparameter tuning.

In [21]:
# Configure Sweep Job
sweep_job = job_for_sweep.sweep(
    compute="cpu-cluster-E2E",
    sampling_algorithm="random",
    primary_metric="Mean Squared Error",
    goal="Minimize",
    max_total_trials=4,
    max_concurrent_trials=1,
)

### ** Run the Sweep Job **

In [22]:
# Create or update the Sweep Job
returned_sweep_job = ml_client.create_or_update(sweep_job) 

In [23]:
# Stream the output and wait until the Job is finished
ml_client.jobs.stream(returned_sweep_job.name)

RunId: hungry_kitten_rqmnbzg23l
Web View: https://ml.azure.com/runs/hungry_kitten_rqmnbzg23l?wsid=/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourcegroups/JuvlinResourceGroup/workspaces/JuvlinWorkspace

Streaming azureml-logs/hyperdrive.txt

[2024-11-13T13:34:34.6656382Z][GENERATOR][DEBUG]Sampled 1 jobs from search space 
[2024-11-13T13:34:34.8704292Z][SCHEDULER][INFO]Scheduling job, id='hungry_kitten_rqmnbzg23l_0' 
[2024-11-13T13:34:35.4425132Z][SCHEDULER][INFO]Successfully scheduled a job. Id='hungry_kitten_rqmnbzg23l_0' 
[2024-11-13T13:36:06.5389786Z][GENERATOR][DEBUG]Sampled 1 jobs from search space 
[2024-11-13T13:36:06.7296989Z][SCHEDULER][INFO]Scheduling job, id='hungry_kitten_rqmnbzg23l_1' 
[2024-11-13T13:36:07.0042169Z][SCHEDULER][INFO]Successfully scheduled a job. Id='hungry_kitten_rqmnbzg23l_1' 
[2024-11-13T13:37:08.1603650Z][GENERATOR][DEBUG]Sampled 1 jobs from search space 
[2024-11-13T13:37:08.3699378Z][SCHEDULER][INFO]Scheduling job, id='hungry_kitten_rqmnbzg23

In [24]:
# Refresh the latest status of the Job after Streaming
returned_sweep_job = ml_client.jobs.get(name=returned_sweep_job.name)

### ** Extract the run that gave Best Modeling results **

In [25]:
# Find the Best Model
from azure.ai.ml.entities import Model

if returned_sweep_job.status == "Completed":

    # Get the run with the best result
    best_run = returned_sweep_job.properties["best_child_run_id"]

    # Get the Model from that run
    model = Model(
        # Stores the Best Model as "milestone1_best_model"
        path="azureml://jobs/{}/outputs/artifacts/paths/milestone1_model/".format(
            best_run
        ),
        name="milestone1_best_model",
        description="Model created from Advertising for Milestone1 Project",
        type="custom_model",
    )

else:
    print(
        "Sweep job status: {}. Please wait until it completes".format(
            returned_sweep_job.status
        )
    )

In [27]:
print(f"Best Model: {model.name}")
print(f"Best Model Path: {model.path}")
print(f"Best Model Type: {model.type}")

Best Model: milestone1_best_model
Best Model Path: azureml://jobs/hungry_kitten_rqmnbzg23l_2/outputs/artifacts/paths/milestone1_model/
Best Model Type: custom_model


### ** Register the Best Model **

In [28]:
# Register the Best Model
registered_model = ml_client.models.create_or_update(model=model)
print(f"Best Model: {registered_model.name}")
print(f"Best Model Path: {registered_model.path}")

Best Model: milestone1_best_model
Best Model Path: azureml://subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourceGroups/JuvlinResourceGroup/workspaces/JuvlinWorkspace/datastores/workspaceartifactstore/paths/ExperimentRun/dcid.hungry_kitten_rqmnbzg23l_2/milestone1_model


### ** Configure an Endpoint **

In [29]:
# Import required Libraries
from azure.ai.ml.entities import (
    ManagedOnlineEndpoint,
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)
from azure.ai.ml.constants import AssetTypes

In [30]:
model_name = "milestone1_best_model"

model = Model(
    name=model_name,
    #The name of the MLflow model.
    path="milestone1_best_model/milestone1_decisiontree_model",
    #Path to the root directory of the model.
    type=AssetTypes.MLFLOW_MODEL,
    #The type of the model asset(MLflow model).
    description="MLflow model for the pima classification problem",
    #The purpose of the model.
)

In [31]:
# Create a unique Endpoint Name
endpoint_suffix = "juvlin"
endpoint_name = "milestone1-endpoint-" + endpoint_suffix
print(f"Endpoint name: {endpoint_name}")

Endpoint name: milestone1-endpoint-juvlin


In [32]:
# Configure the Endpoint
endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,   # Unique Endpoint Name within Deployment
    description="An online endpoint serving an MLflow model, for Milestone1 Advertising Prediction task",
                          # A string describing the purpose of the endpoint
    auth_mode="key",      # Authentication mode for endpoint - API key
    tags={"foo": "bar"},  # Dictionary of key-value pairs to tag the Endpoint
)

### ** Create an Endpoint **

In [33]:
# Create the Endpoint
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

ManagedOnlineEndpoint({'public_network_access': 'Enabled', 'provisioning_state': 'Succeeded', 'scoring_uri': 'https://milestone1-endpoint-juvlin.eastus2.inference.ml.azure.com/score', 'openapi_uri': 'https://milestone1-endpoint-juvlin.eastus2.inference.ml.azure.com/swagger.json', 'name': 'milestone1-endpoint-juvlin', 'description': 'An online endpoint serving an MLflow model, for Milestone1 Advertising Prediction task', 'tags': {'foo': 'bar'}, 'properties': {'createdBy': 'Juvlin Pinheiro', 'createdAt': '2024-11-13T14:12:50.919396+0000', 'lastModifiedAt': '2024-11-13T14:12:50.919396+0000', 'azureml.onlineendpointid': '/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/resourcegroups/juvlinresourcegroup/providers/microsoft.machinelearningservices/workspaces/juvlinworkspace/onlineendpoints/milestone1-endpoint-juvlin', 'AzureAsyncOperationUri': 'https://management.azure.com/subscriptions/6793e723-756c-4c5d-84c0-812f1bb4c679/providers/Microsoft.MachineLearningServices/locations/eastus2/mfe

### ** Create a Deployment Script to perform Model Deployment **

In [34]:
%%writefile {src_dir}/deploymodel.py

import logging
import os
import json
import mlflow
from io import StringIO
from mlflow.pyfunc.scoring_server import infer_and_parse_json_input

# Application Insights Setup
from opencensus.ext.azure.log_exporter import AzureLogHandler

# Configure logging to Application Insights
# app_insights_conn_str = 'InstrumentationKey=e040f1cb-4a7a-4009-87be-7861c813da51;IngestionEndpoint=https://eastus-8.in.applicationinsights.azure.com/;LiveEndpoint=https://eastus.livediagnostics.monitor.azure.com/'
application_insights_connection_string= 'InstrumentationKey=e040f1cb-4a7a-4009-87be-7861c813da51;IngestionEndpoint=https://eastus-8.in.applicationinsights.azure.com/;LiveEndpoint=https://eastus.livediagnostics.monitor.azure.com/'
handler = AzureLogHandler(connection_string=app_insights_conn_str)
logger = logging.getLogger()
logger.addHandler(handler)
logger.setLevel(logging.INFO)

def init():
    global model
    global input_schema

    # Load the model from the Azure ML directory
    model_path = os.path.join(os.getenv("AZUREML_MODEL_DIR"), "milestone1_decisiontree_model")
    model = mlflow.pyfunc.load_model(model_path)
    input_schema = model.metadata.get_input_schema()

def run(raw_data):
    # Parse input JSON data
    json_data = json.loads(raw_data)
    if "input_data" not in json_data:
        raise Exception("Request must contain a top level key named 'input_data'")
    
    # Parse data according to input schema
    serving_input = json_data["input_data"]
    data = infer_and_parse_json_input(json.dumps(serving_input), input_schema)

    # Generate predictions
    predictions = model.predict(data)

    # Log data and predictions
    logger.info("Data:{0},Predictions:{1}".format(str(data),str(predictions)))

    # Convert predictions to JSON and return
    result = StringIO()
    predictions_to_json(predictions, result)
    return result.getvalue()

Overwriting ./src/deploymodel.py


In [43]:
# from azure.ai.ml.entities import CodeConfiguration, ManagedOnlineDeployment, OnlineRequestSettings

# # Set up the request settings correctly using OnlineRequestSettings
# request_settings = OnlineRequestSettings(
#     request_timeout_ms=180000,  # 20 minutes in milliseconds
#     max_concurrent_requests_per_instance=1,
#     max_queue_wait_ms=600000  # 10 minutes in milliseconds
# )
# Create a new Deployment with name "blue"
blue_deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,                  # Use the previously generated endpoint name
    model=registered_model,                       # Use the registered model
    environment="Milestone1_Project_E2E@latest",  # Use the latest environment"
    # Use the code in the "./src" directory and the "score.py" script
    code_configuration=CodeConfiguration(
        code="./src", scoring_script="deploymodel.py"
    ),
    instance_type="Standard_E2s_v3",              # Use a single instance of type "Standard_E2s_v3"
    # instance_type="Standard_F4s_v2",
    # instance_type="Standard_D2s_v3",
    # instance_type="Standard_F2s_v2",
    instance_count=1,
    app_insights_enabled=True,                    # Enable Application Insights for the deployment
    # request_settings=request_settings,  # Pass the OnlineRequestSettings instance
    # request_settings=request_settings,
    # liveness_probe=liveness_probe_settings  # Add the liveness probe settings
)

In [44]:
# print(f"Model Path: {model_path}")
# print(f"Model Name: {model.name}")

In [45]:
# Deploy the Model to Endpoint
ml_client.online_deployments.begin_create_or_update(blue_deployment).result()

Check: endpoint milestone1-endpoint-juvlin exists


..............................

HttpResponseError: (ResourceNotReady) User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready
Code: ResourceNotReady
Message: User container has crashed or terminated. Please see troubleshooting guide, available here: https://aka.ms/oe-tsg#error-resourcenotready

In [ ]:
# Print the endpoint and deployment names to verify they are correctly assigned
print("Endpoint Name:", endpoint_name)
print("Deployment Name:", "blue")
# print("Endpoint invocation response:", response)

Inspect Logs for Specific Errors:

In [ ]:
ml_client.online_deployments.get_logs(name="blue", endpoint_name=endpoint_name)

### ** Delete the Endpoint **

**Important!** An Endpoint is a LIVE node which is always running, ready to process & predict to give you output. So unless you are making real-time predictions on streaming data, delete your endpoints after use

In [ ]:
# Delete Endpoint
# ml_client.online_endpoints.begin_delete(name=endpoint_name)

### 